In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/My_model/pytorch-forecasting-master')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

!pip uninstall tensorflow
!pip install pytorch_lightning
!pip install optuna
!pip install plotly

Mounted at /content/drive
Tue Jan  3 06:06:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                              

In [2]:
#from ctypes import FormatError
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import os,sys

# sys.path.append(os.path.abspath(os.path.join('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\\2_freq_nbinom_LSTM')))

# sys.path.append(os.path.abspath(os.path.join('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\\2_freq_nbinom_LSTM\\1_cluster_demand_prediction\data\weather_data')))
# sys.path.append(os.path.abspath(os.path.join('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\2_freq_nbinom_LSTM\1_cluster_demand_prediction\data\demand_data')))

import torch
torch.use_deterministic_algorithms(True)

from pytorch_forecasting.data.encoders import TorchNormalizer
from pytorch_forecasting.metrics import SMAPE, RMSE
from torchmetrics import R2Score, SymmetricMeanAbsolutePercentageError, MeanSquaredError

import matplotlib.pyplot as plt
import pandas as pd
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
import pytorch_lightning as pl
import torch
from pytorch_forecasting.data.encoders import TorchNormalizer
import os,sys
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import acf,pacf
from scipy.signal import find_peaks
import operator
import statsmodels.api as sm
from itertools import combinations
import pickle
from pytorch_forecasting import Baseline
import random
from pytorch_forecasting import DeepAR,NegativeBinomialDistributionLoss
from itertools import product
from sklearn.metrics import mean_absolute_error, mean_squared_error
import optuna
from optuna.trial import TrialState
import plotly

In [3]:
"""
Set Random seed
"""

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
## additional seeding to ensure reproduciblility.
pl.seed_everything(0)

import os
os.chdir('/content/drive/MyDrive/My_model/pytorch-forecasting-master/1_cluster_demand_prediction')

INFO:lightning_lite.utilities.seed:Global seed set to 0


In [4]:
encoder_len = 8
pred_len = 1 # 1, 6, 12 
#cov_lag_len= pred_len
cov_lag_len= 0 #we can use forecasted values, even for inflow

In [5]:
(20+cov_lag_len)%24

20

In [6]:
"""
Import pre-processed Data

response and target are the same thing
"""
with open("final_response_ts_tampines_medium_dataset_lag_502.pkl",'rb') as f:
    final_response_ts = pickle.load(f)

with open("final_num_covariate_ts_tampines_medium_dataset_lag_502.pkl",'rb') as f:
    final_num_covariate_ts = pickle.load(f)

with open("final_cat_covariate_ts_tampines_medium_dataset_lag_502.pkl",'rb') as f:
    final_cat_covariate_ts = pickle.load(f)

###### concatenate response and covariate time series into a single dataframe  #########
df3 = pd.DataFrame(final_response_ts) # response and target are the same thing
# Remove the excess cov_lag_len hours for lagging covariate time series by 24 hours
#df3 = pd.DataFrame(df3.iloc[cov_lag_len:]).reset_index(drop=True)

df4 = pd.DataFrame(final_num_covariate_ts)
# lag covariate time series by cov_lag_len hours to turn it into a 'time_varying_unknown_real' variable
#df4 = pd.DataFrame(df4.iloc[:-cov_lag_len]).reset_index(drop=True)

df5 = pd.DataFrame(final_cat_covariate_ts)
# lag covariate time series by cov_lag_len hours to turn it into a 'time_varying_unknown_real' variable
#df5 = pd.DataFrame(df5.iloc[:-cov_lag_len]).reset_index(drop=True)

df6 = pd.concat([df5,df4], axis=1) # dataframe of all covariates

df1 = pd.DataFrame({"time_idx":np.arange(0,len(df3))})
df2 = pd.DataFrame({"group":np.tile(0,len(df3))}) # identifies the series in case of univariate and multivariate series
df = pd.concat([df1,df2,df3,df6], axis=1)
###### concatenate response and covariate time series into a single dataframe  #########


#################### add date information ts ####################
#2021 oct 17 20:00:00
day=17
hour=(20+cov_lag_len)
if (20+cov_lag_len)>23:
  day = 18
  hour = hour%24

df["date"] = pd.Timestamp(year=2021, month=10, day=day, hour=hour ) + pd.to_timedelta(df.time_idx, "H")
df['_hour_of_day'] = df["date"].dt.hour.astype(str)
df['_day_of_week'] = df["date"].dt.dayofweek.astype(str)
df['_day_of_month'] = df["date"].dt.day.astype(str)
df['_day_of_year'] = df["date"].dt.dayofyear.astype(str)
df['_week_of_year'] = df["date"].dt.weekofyear.astype(str)
df['_month_of_year'] = df["date"].dt.month.astype(str)
df['_year'] = df["date"].dt.year.astype(str)
#################### add date information ts ####################

Target = list(final_response_ts.keys())[0] # response and target are the same thing
chosen = Target[6:]

tr_stop_idx = int(0.7*len(df3))
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = val_start_idx + int(0.2*len(df3)) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
tes_start_idx = val_stop_idx - encoder_len
tes_stop_idx = len(df3)

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
print(tes_start_idx)
print(tes_stop_idx)

cat_col1 = list(final_cat_covariate_ts.keys())[0] # for column name agnostic
# cat_col2 = list(final_cat_covariate_ts.keys())[1] # for column name agnostic
# cat_col3 = list(final_cat_covariate_ts.keys())[2] # for column name agnostic
# cat_col4 = list(final_cat_covariate_ts.keys())[3] # for column name agnostic

#num_cols_list = list(final_num_covariate_ts.keys())[:3] # for column name agnostic
#num_cols_list.append(Target) # for column name agnostic

1117
1109
1428
1420
1596


In [7]:
print(list(df6.columns))

['wea_desc_clstr_175', 'temp_clstr_175', 'hum_clstr_175', 'clstr_171', 'clstr_168', 'clstr_175_lag_502', 'clstr_175_lag_166', 'clstr_175_lag_334', '175_inflow', '175_lag_502_inflow', '175_lag_166_inflow', '175_lag_334_inflow']


In [8]:
cov = list(df6.columns)
cov = cov[:-7] # remove one extra covariate to reduce hyperparam comb. However, these are added again below for every hparams comb.
print(cov)

cov_pairs_list = []
for p in combinations(cov,2):
  cov_pairs_list.append(p)

['wea_desc_clstr_175', 'temp_clstr_175', 'hum_clstr_175', 'clstr_171', 'clstr_168']


In [9]:
cov_pairs_list

[('wea_desc_clstr_175', 'temp_clstr_175'),
 ('wea_desc_clstr_175', 'hum_clstr_175'),
 ('wea_desc_clstr_175', 'clstr_171'),
 ('wea_desc_clstr_175', 'clstr_168'),
 ('temp_clstr_175', 'hum_clstr_175'),
 ('temp_clstr_175', 'clstr_171'),
 ('temp_clstr_175', 'clstr_168'),
 ('hum_clstr_175', 'clstr_171'),
 ('hum_clstr_175', 'clstr_168'),
 ('clstr_171', 'clstr_168')]

In [10]:
"""
set inputs here
(hyperparameters grid search)

"""
######### Network Architecture ###################
p = 10 # patience no. of epochs

Loss=NegativeBinomialDistributionLoss()

######### Network Architecture ###################


######### Training Routine ###################
fdv_steps = 10 # fast_dev_run
######### Training Routine ###################


############## Inputs for 2) Persistance model ( seasonal naive forecast ) #######################
season_len = 168 # length of season
num_past_seas = 6 # number of past seasons to use in averaging
#seas_pred_strt_idx = 2035 # seasonal naive forecast start index, in hours use the df dataframe
############## Inputs for 2) Persistance model ( seasonal naive forecast ) #######################


In [11]:
"""
Data loading sanity check

"""

######## View the entire data using this code ##################
#print(train_dataset.data['categoricals'])
######## View the data entire using this code ##################

######## View the data for each batch using this code ##################
# i=0
# for x,y in iter(train_dataloader):
#     i+=1
#     #print(x["encoder_cat"][0][0:72,2])
#     #break
######## View the data for each batch using this code ##################
#print(i)

'\nData loading sanity check\n\n'

In [12]:
df = df.drop(columns=['175_lag_502_inflow','175_lag_166_inflow','175_lag_334_inflow'])
df.columns

Index(['time_idx', 'group', 'clstr_175', 'wea_desc_clstr_175',
       'temp_clstr_175', 'hum_clstr_175', 'clstr_171', 'clstr_168',
       'clstr_175_lag_502', 'clstr_175_lag_166', 'clstr_175_lag_334',
       '175_inflow', 'date', '_hour_of_day', '_day_of_week', '_day_of_month',
       '_day_of_year', '_week_of_year', '_month_of_year', '_year'],
      dtype='object')

In [13]:
df.columns

Index(['time_idx', 'group', 'clstr_175', 'wea_desc_clstr_175',
       'temp_clstr_175', 'hum_clstr_175', 'clstr_171', 'clstr_168',
       'clstr_175_lag_502', 'clstr_175_lag_166', 'clstr_175_lag_334',
       '175_inflow', 'date', '_hour_of_day', '_day_of_week', '_day_of_month',
       '_day_of_year', '_week_of_year', '_month_of_year', '_year'],
      dtype='object')

In [ ]:
"""
Full Training Routine 
with hyperparmeter grid search

Load data into TimeSeriesDataSet object

for fast development run
uncomment fast_dev_run = fdv_steps

"""

#early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=p, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

RMSE_list = [] # FIND minimum RMSE case
hyperparams_list = [] # FIND minimum RMSE case

# best_val_comb_idx=[17,21,51,52,53,54,61,62,63,82,83,84,109,110,111,143,144,145,195,218,219,220,232,233,234,236,237,238,280,338,339,340,344,345,346,386]
# best_val_train_epochs = [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]

# best_val_comb_idx=[234]
# best_val_train_epochs = [50]


param_comb_cnt=-1
#for neu,lay,bat,lr,enc_len,pred_len,drop,cov_pair,num_ep in product(*[x for x in hparams_grid.values()]):

def objective(trial, ):  
  
  neu = trial.suggest_int(name="neu",low=500,high=700,step=10,log=False)
  lay = trial.suggest_int(name="lay",low=1,high=3,step=1,log=False)
  bat = trial.suggest_int(name="bat",low=4,high=8,step=4,log=False)
  lr = trial.suggest_float(name="lr",low=0.00001,high=0.0008,step=0.00001,log=False)
  enc_len = 8
  pred_len = 1
  drop = trial.suggest_float(name="dropout",low=0,high=0.2,step=0.2,log=False)
  cov_pair = trial.suggest_categorical("cov_pair", cov_pairs_list)

  for num_ep in range(30,50,3):

    num_cols_list = []  
    #param_comb_cnt+=1
    # if param_comb_cnt <0 or param_comb_cnt > 40:
    # #if param_comb_cnt not in [257, 254, 258, 291, 294, 295,328, 322, 326, 327,337, 329]:
    #   continue

    df_cov_col1 = cov_pair[0]
    df_cov_col2 = cov_pair[1]

      ######### Load DATA #############

    if (df_cov_col1[:3] == 'wea' and df_cov_col2[:3] != 'wea'): 
          num_cols_list = [df_cov_col2]
          cat_col1 = df_cov_col1
          cat_dict = {cat_col1: NaNLabelEncoder(add_nan=True).fit(df[cat_col1]),"_hour_of_day": NaNLabelEncoder(add_nan=True).fit(df._hour_of_day), \
          "_day_of_week": NaNLabelEncoder(add_nan=True).fit(df._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(df._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(df._day_of_year), \
              "_week_of_year": NaNLabelEncoder(add_nan=True).fit(df._week_of_year), "_month_of_year": NaNLabelEncoder(add_nan=True).fit(df._month_of_year) ,"_year": NaNLabelEncoder(add_nan=True).fit(df._year)}
          cat_list = [cat_col1,"_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_month_of_year","_year"]
          embed_size_dict = {cat_col1:(18,4),'_hour_of_day':(25,5),'_day_of_week':(8,5),'_day_of_month':(32,5), '_day_of_year':(69,8), '_week_of_year':(12,3), '_month_of_year':(4,3),'_year':(2,1)  }
  
    elif (df_cov_col2[:3] == 'wea' and df_cov_col1[:3] != 'wea'): 
          num_cols_list = [df_cov_col1]
          cat_col1 = df_cov_col2
          cat_dict = {cat_col1: NaNLabelEncoder(add_nan=True).fit(df[cat_col1]),"_hour_of_day": NaNLabelEncoder(add_nan=True).fit(df._hour_of_day), \
          "_day_of_week": NaNLabelEncoder(add_nan=True).fit(df._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(df._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(df._day_of_year), \
              "_week_of_year": NaNLabelEncoder(add_nan=True).fit(df._week_of_year), "_month_of_year": NaNLabelEncoder(add_nan=True).fit(df._month_of_year) ,"_year": NaNLabelEncoder(add_nan=True).fit(df._year)}
          cat_list = [cat_col1,"_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_month_of_year","_year"]
          embed_size_dict = {cat_col1:(18,4),'_hour_of_day':(25,5),'_day_of_week':(8,5),'_day_of_month':(32,5), '_day_of_year':(69,8), '_week_of_year':(12,3), '_month_of_year':(4,3),'_year':(2,1)  }
    
    elif (df_cov_col1[:3] == 'wea' and df_cov_col2[:3] == 'wea'): 
          num_cols_list = []
          cat_col1 = df_cov_col1
          cat_col2 = df_cov_col2
          cat_dict = {cat_col1: NaNLabelEncoder(add_nan=True).fit(df[cat_col1]),cat_col2: NaNLabelEncoder(add_nan=True).fit(df[cat_col2]),"_hour_of_day": NaNLabelEncoder(add_nan=True).fit(df._hour_of_day), \
          "_day_of_week": NaNLabelEncoder(add_nan=True).fit(df._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(df._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(df._day_of_year), \
              "_week_of_year": NaNLabelEncoder(add_nan=True).fit(df._week_of_year), "_month_of_year": NaNLabelEncoder(add_nan=True).fit(df._month_of_year) ,"_year": NaNLabelEncoder(add_nan=True).fit(df._year)}
          cat_list = [cat_col1,cat_col2,"_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_month_of_year","_year"]
          embed_size_dict = {cat_col1:(18,4),'_hour_of_day':(25,5),'_day_of_week':(8,5),'_day_of_month':(32,5), '_day_of_year':(69,8), '_week_of_year':(12,3), '_month_of_year':(4,3),'_year':(2,1)  }
    
    else:
          num_cols_list = [df_cov_col1,df_cov_col2] # numerical covariate
          cat_dict = {"_hour_of_day": NaNLabelEncoder(add_nan=True).fit(df._hour_of_day), \
          "_day_of_week": NaNLabelEncoder(add_nan=True).fit(df._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(df._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(df._day_of_year), \
              "_week_of_year": NaNLabelEncoder(add_nan=True).fit(df._week_of_year), "_month_of_year": NaNLabelEncoder(add_nan=True).fit(df._month_of_year) ,"_year": NaNLabelEncoder(add_nan=True).fit(df._year),\
              'wea_desc_clstr_'+chosen: NaNLabelEncoder(add_nan=True).fit(df['wea_desc_clstr_'+chosen]) }
          cat_list = ["_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_month_of_year","_year"]
          embed_size_dict = {'_hour_of_day':(25,5),'_day_of_week':(8,5),'_day_of_month':(32,5), '_day_of_year':(69,8), '_week_of_year':(12,3), '_month_of_year':(4,3),'_year':(2,1)  }

    #num_cols_list.append('clstr_'+chosen+'_lag_22')  
    num_cols_list.append('clstr_'+chosen+'_lag_166') 
    num_cols_list.append('clstr_'+chosen+'_lag_334')
    num_cols_list.append('clstr_'+chosen+'_lag_502')
    num_cols_list.append(chosen+'_inflow')
    #num_cols_list.append(chosen+'_lag_22_inflow') 
    # num_cols_list.append(chosen+'_lag_166_inflow')  
    # num_cols_list.append(chosen+'_lag_334_inflow')
    # num_cols_list.append(chosen+'_lag_502_inflow')  
    #cat_list.append('wea_desc_clstr_52')

    train_dataset = TimeSeriesDataSet(
        df.iloc[0:tr_stop_idx],
        time_idx="time_idx",
        target=Target,
        categorical_encoders=cat_dict,
        group_ids=["group"],
        min_encoder_length=enc_len,
        max_encoder_length=enc_len,
        min_prediction_length=pred_len,
        max_prediction_length=pred_len,
        time_varying_unknown_reals=[Target],
        time_varying_known_reals=num_cols_list,
        time_varying_known_categoricals=cat_list,
        add_relative_time_idx=False,
        randomize_length=False,
        scalers={},
        target_normalizer=TorchNormalizer(method="identity",center=False,transformation=None )

    )

    val_dataset = TimeSeriesDataSet.from_dataset(train_dataset,df.iloc[val_start_idx:val_stop_idx], stop_randomization=True, predict=False)
    test_dataset = TimeSeriesDataSet.from_dataset(train_dataset,df.iloc[tes_start_idx:tes_stop_idx], stop_randomization=True)

    train_dataloader = train_dataset.to_dataloader(train=True, batch_size=bat)
    val_dataloader = val_dataset.to_dataloader(train=False, batch_size=bat)
    test_dataloader = test_dataset.to_dataloader(train=False, batch_size=bat)
    ######### Load DATA #############


    """
    Machine Learning predictions START
    1) DeepAR

    """
    trainer = pl.Trainer(
        max_epochs=num_ep,
        gpus=-1, #-1
        auto_lr_find=False,
        gradient_clip_val=0.1,
        limit_train_batches=1.0,
        limit_val_batches=1.0,
        #fast_dev_run=fdv_steps,
        logger=True,
        #log_every_n_steps=10,
        # profiler=True,
        callbacks=[lr_logger]#, early_stop_callback],
        #enable_checkpointing=True,
        #default_root_dir="C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\2_freq_nbinom_LSTM\1_cluster_demand_prediction\logs"
    )


    #print(f"training routing:\n \n {trainer}")
    deepar = DeepAR.from_dataset(
        train_dataset,
        learning_rate=lr,
        hidden_size=neu,
        rnn_layers=lay,
        dropout=drop,
        loss=Loss,
        log_interval=20,
        log_val_interval=6,
        log_gradient_flow=False,
        embedding_sizes=embed_size_dict
        # reduce_on_plateau_patience=3,
    )

    #print(f"Number of parameters in network: {deepar.size()/1e3:.1f}k")
    # print(f"Model :\n \n {deepar}")
    torch.set_num_threads(10)
    trainer.fit(
        deepar,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    val_rmse = trainer.callback_metrics['val_RMSE'].item()
    trial.report(val_rmse, num_ep)

    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

  return val_rmse








########## optuna results #####################
if __name__ == "__main__":

  study = optuna.create_study(direction="minimize")
  study.optimize(objective, timeout=6000)

  pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
  complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

  print("Study statistics: ")
  print("  Number of finished trials: ", len(study.trials))
  print("  Number of pruned trials: ", len(pruned_trials))
  print("  Number of complete trials: ", len(complete_trials))

  print("Best trial:")
  trial = study.best_trial

  print("  Value: ", trial.value)

  print("  Params: ")
  for key, value in trial.params.items():
      print("    {}: {}".format(key, value))

  fig = optuna.visualization.plot_parallel_coordinate(study)
  fig.show()

  fig = optuna.visualization.plot_optimization_history(study)
  fig.show()

  fig = optuna.visualization.plot_slice(study)
  fig.show()

  fig = optuna.visualization.plot_param_importances(study)
  fig.show()
########## optuna results #####################






[I 2023-01-03 10:37:05,762] A new study created in memory with name: no-name-25ebac70-bd36-454f-8c57-13ba55ca5f28
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]